In [ ]:
import os
import gc
from scipy.stats import pearsonr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from scipy.stats import norm, skew, kurtosis
from scipy import stats
import datatable as dt
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import plotly.express as px
from scipy import stats
from scipy.stats import norm, skew, kurtosis
import gc
import warnings
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from scipy import stats
from tensorflow.python.ops import math_ops
from tensorflow.python.keras import backend as K
from scipy.special import comb
from itertools import combinations
warnings.filterwarnings('ignore')

**Problem Definition: **

The aim of this competition is to predict obfuscated (unclear) metric which is relevant for making trading decisions from the features derived from real historic data from thousands of investments.

There are 3579 unique investments in training set, but investments in training set doesn't necessarily appear in public or private test set. There are also 1211 unique time IDs in training set. All of the investments doesn't necessarily appear in all time IDs.

**Columns of the train data-**

row_id - A unique identifier for the row. 

time_id - The ID code for the time the data was gathered. The time IDs are in order, but the real time between the time IDs is not constant and will likely be shorter for the final private test set than in the training set.

investment_id - The ID code for an investment. Not all investment have data in all time IDs.

target - The target.

[f_0:f_299] - Anonymized features generated from market data.

## **The goal of this competition is to build a model that forecasts an investment's return rate.**

Our submission for this project is being scored based on the mean of Pearson correlation coefficient for each time ID. We would like to measure our success by comparing our score with the scores of top performing teams on the Leaderboard. By the end of this kaggle project, we aim to score high and keep ourselves on top of the leaderboard.

# Reading the Data

In [ ]:
# df = pd.read_parquet('../input/low-mem-parquet/train_low_mem.parquet')
# train=df[:20000]
# df2=df
# df.drop(['row_id'],axis=1, inplace=True)
# #test=pd.read_csv('../input/ubiquant-market-prediction/example_test.csv')

In [ ]:
# class CombinatorialPurgedGroupKFold():
#     def __init__(self, n_splits = 6, n_test_splits = 2, purge = 1, pctEmbargo = 0.01, **kwargs):
#         self.n_splits = n_splits
#         self.n_test_splits = n_test_splits
#         self.purge = purge
#         self.pctEmbargo = pctEmbargo
        
#     def split(self, X, y = None, groups = None):
#         if groups is None:
#             raise ValueError(
#                 "The 'groups' parameter should not be None")
            
#         u, ind = np.unique(groups, return_index = True)
#         unique_groups = u[np.argsort(ind)]
#         n_groups = len(unique_groups)
#         group_dict = {}
#         for idx in range(len(X)):
#             if groups[idx] in group_dict:
#                 group_dict[groups[idx]].append(idx)
#             else:
#                 group_dict[groups[idx]] = [idx]
                
#         n_folds = comb(self.n_splits, self.n_test_splits, exact = True)
#         if n_folds > n_groups:
#             raise ValueError(
#                 ("Cannot have number of folds={0} greater than"
#                  " the number of groups={1}").format(n_folds,
#                                                      n_groups))
            
#         mbrg = int(n_groups * self.pctEmbargo)
#         if mbrg < 0:
#             raise ValueError(
#                 "The number of 'embargoed' groups should not be negative")
        
#         split_dict = {}
#         group_test_size = n_groups // self.n_splits
#         for split in range(self.n_splits):
#             if split == self.n_splits - 1:
#                 split_dict[split] = unique_groups[int(split * group_test_size):].tolist()
#             else:
#                 split_dict[split] = unique_groups[int(split * group_test_size):int((split + 1) * group_test_size)].tolist()
        
#         for test_splits in combinations(range(self.n_splits), self.n_test_splits):
#             test_groups = []
#             banned_groups = []
#             for split in test_splits:
#                 test_groups += split_dict[split]
#                 banned_groups += unique_groups[split_dict[split][0] - self.purge:split_dict[split][0]].tolist()
#                 banned_groups += unique_groups[split_dict[split][-1] + 1:split_dict[split][-1] + self.purge + mbrg + 1].tolist()
#             train_groups = [i for i in unique_groups if (i not in banned_groups) and (i not in test_groups)]

#             train_idx = []
#             test_idx = []
#             for train_group in train_groups:
#                 train_idx += group_dict[train_group]
#             for test_group in test_groups:
#                 test_idx += group_dict[test_group]
#             yield train_idx, test_idx

In [ ]:
# investment_id = train.pop("investment_id")
# time_id = train.pop("time_id")
# y = train.pop("target")
# def create_record(i):
#     dic = {}
#     dic[f"features"] = tf.train.Feature(float_list=tf.train.FloatList(value=list(train.iloc[i])))
#     dic["time_id"] = tf.train.Feature(int64_list=tf.train.Int64List(value=[time_id.iloc[i]]))
#     dic["investment_id"] = tf.train.Feature(int64_list=tf.train.Int64List(value=[investment_id.iloc[i]]))
#     dic["target"] = tf.train.Feature(float_list=tf.train.FloatList(value=[y.iloc[i]]))
#     record_bytes = tf.train.Example(features=tf.train.Features(feature=dic)).SerializeToString()
#     return record_bytes
    
# def decode_function(record_bytes):
#   return tf.io.parse_single_example(
#       # Data
#       record_bytes,
#       # Schema
#       {
#           "features": tf.io.FixedLenFeature([300], dtype=tf.float32),
#           "time_id": tf.io.FixedLenFeature([], dtype=tf.int64),
#           "investment_id": tf.io.FixedLenFeature([], dtype=tf.int64),
#           "target": tf.io.FixedLenFeature([], dtype=tf.float32)
#       }
#   )

In [ ]:
# %%time
# import time
# n_splits = 5
# n_test_splits = 1
# kfold = CombinatorialPurgedGroupKFold(n_splits, n_test_splits)
# for fold, (train_indices, test_indices) in enumerate(kfold.split(train, groups=time_id)):
#     print(f"Fold")
#     print("=" * 100)
#     print("Train Sample size:", len(train_indices))
#     print("Test Sample size:", len(test_indices))
#     train_save_path = f"fold_{fold}_train.tfrecords"
#     begin = time.time()
#     print(f"Creating {train_save_path}")
#     with tf.io.TFRecordWriter(train_save_path) as file_writer:
#         for i in train_indices:
#             file_writer.write(create_record(i))
#     print("Elapsed time: %.2f"%(time.time() - begin))
#     begin = time.time()
#     print(f"Creating {train_save_path}")
#     test_save_path = f"fold_{fold}_test.tfrecords"
#     with tf.io.TFRecordWriter(test_save_path) as file_writer:
#         for i in test_indices:
#             file_writer.write(create_record(i))
#     print("Elapsed time: %.2f"%(time.time() - begin))

In [ ]:
# investment_ids = investment_id.unique()
# investment_id_df = pd.DataFrame({"investment_id": investment_ids})
# investment_id_df.to_csv("investment_ids.csv", index=False)

# Test of Null Values

Checking null values in the dataframe. As the output of the below mentioned snippet is False, with which we can conclude that there are no null values in the dataset.

In [ ]:
#print(df.isnull().any().any()) #No Null Values

# Test of Outliers

In [ ]:
#df['f_0'].plot.box(figsize=(100,100))

In [ ]:
# for i in df.columns[3:]:       #Test of Outlier
#     print(pd.DataFrame(df[i].describe()))

# Checking the category of Columns

In [ ]:
# print(df.dtypes)
# print(pd.DataFrame(df.dtypes=='float64').count() if (df.dtypes=='float64').any() else "None")
# print((df.dtypes=='float64'))

In [ ]:
# df['investment_id'].describe()

In [ ]:
# df['target'].describe()

In [ ]:
# features = [f'f_{i}' for i in range(300)]
# for f in features:
#     df[f] = df[f].astype('float16')

# 1) Univariate Analysis 

## 1 a) Target and it's frequency count 

In [ ]:
# f = plt.figure(figsize=(16, 10))
# ax=sns.histplot(df['target'].values, kde=True, color='Red')
# ax.set()
# ax.set_xlabel('Target',fontsize=20)
# ax.set_ylabel('Frequency',fontsize=20)
# plt.show()

## 1 b) Investment ID and it's frequency count 

In [ ]:
# f = plt.figure(figsize=(16, 10))
# ax = sns.histplot(x="investment_id", data=df, color='gray')
# ax.set_xlabel('Investment ID',fontsize=20)
# ax.set_ylabel('Frequency',fontsize=20)
# plt.show()

## 1 c) features and it's frequency count which doesn't follow Gaussian distibution

In [ ]:
# def feat_dist(df, cols, rows=3, columns=3, title=None, figsize=(30, 25)):
    
#     '''A function for displaying skew feat distribution'''
    
#     fig, axes = plt.subplots(rows, columns, figsize=figsize, constrained_layout=True)
#     axes = axes.flatten()

#     for i, j in zip(cols, axes):
#         sns.distplot(
#                     df[i],
#                     ax=j,
#                     fit=norm,
#                     hist=False,
#                     kde_kws={'linewidth':3}
#         )   
        
#         (mu, sigma) = norm.fit(df[i])
#         j.set_title('Dist of {0} Norm Fit: $\mu=${1:.2g}, $\sigma=${2:.2f}'.format(i, mu, sigma), weight='bold')
#         j.legend(labels=[f'{i}', 'Normal Dist'])
#         fig.suptitle(f'{title}', fontsize=24, weight='bold')

In [ ]:
# sampled_df = df.sample(frac=0.05, random_state=42)
# features_std = sampled_df.iloc[:,4:].apply(lambda x: np.std(x)).sort_values(
#     ascending=False)
# f_std = sampled_df[features_std.iloc[:25].index.tolist()]

# with pd.option_context('mode.use_inf_as_na', True):
#     features_skew = np.abs(sampled_df.iloc[:,4:].apply(lambda x: np.abs(skew(x))).sort_values(
#         ascending=False)).dropna()
# skewed = sampled_df[features_skew.iloc[:25].index.tolist()]

# with pd.option_context('mode.use_inf_as_na', True):
#     features_kurt = np.abs(sampled_df.iloc[:,4:].apply(lambda x: np.abs(kurtosis(x))).sort_values(
#         ascending=False)).dropna()
# kurt_f = sampled_df[features_kurt.iloc[:25].index.tolist()]


# feat_dist(sampled_df, f_std.columns.tolist(), rows=1, columns=2, title='Distribution of High Std Features', figsize=(30, 8))

In [ ]:
# feat_dist(sampled_df, skewed.columns.tolist(), rows=5, columns=5, title='Distribution of Skewed Features')

### 1d) Highly Correlated features

In [ ]:
# correlations = sampled_df.corrwith(sampled_df['target']).iloc[:-1].to_frame()
# correlations['Abs Corr'] = correlations[0].abs()
# sorted_correlations = correlations.sort_values('Abs Corr', ascending=False)['Abs Corr']
# fig, ax = plt.subplots(figsize=(15,15))
# sns.heatmap(sorted_correlations.iloc[1:].to_frame()[sorted_correlations>=.046], cmap='BuPu', annot=True, vmin=0.046, vmax=0.067, ax=ax)
# plt.title('Feature Correlations With Target')
# plt.savefig('correlation.jpg')
# plt.show()

# 2) HeatMap

In [ ]:
# df_temp = df
# corr = df_temp.corr()
# f = plt.figure(figsize=(100, 100))
# sns.heatmap(corr)

# 3) Bivariate Analysis

## 3 a) Investment_ID vs Target

In [ ]:
# import pylab as pl
# df_temp=df.set_index('time_id')
# for f in np.random.choice(df['investment_id'].unique(),25):
#     ax=df[df['investment_id'] == f]['target'].hist(bins = 100, alpha = 0.2, figsize = (20,10))
#     ax.set_xlabel("Target", fontsize=20)
#     ax.set_ylabel("Investment_id", fontsize=20)
#     ax.tick_params(axis='both', which='major', labelsize=15)

## 3 b) Increase in Investement along the time horizon

In [ ]:
# fig, ax = plt.subplots(1, 1, figsize=(12, 6))
# df.groupby('time_id')['investment_id'].nunique().plot(color='magenta')
# plt.title("number of unique assets by time")
# plt.xlabel('Time_ID', size=20)
# plt.ylabel('Frequency', size=20)
# plt.show()

## 3 c) Plot of Time ID vs Target for set of investment ID

In [ ]:
# df_temp=df[['investment_id','time_id','target']]
# for f in np.random.choice(df_temp['investment_id'].unique(),5):
#     d = df_temp[df_temp['investment_id'] == f]
#     d.set_index('time_id')['target'].plot(figsize=(15, 5),style='.-',label='investment_id'+str(f))
#     plt.xlabel('Time_ID', size=20)
#     plt.ylabel('Target', size=20)
#     plt.show()

## Investment Count vs target dispersion

In [ ]:
# obs_by_asset = df.groupby(['investment_id'])['target'].count()
# mean_target = df.groupby(['investment_id'])['target'].mean()
# ax = sns.jointplot(x=obs_by_asset, y=mean_target, kind="reg", 
#                    height=8, joint_kws={'line_kws':{'color':'green'}})
# ax.ax_joint.set_xlabel('Count by investment ID', size=20)
# ax.ax_joint.set_ylabel('Target by mean', size=20)
# plt.show()

## Investment Count vs target dispersion using Standard Deviation

In [ ]:
# sts_target = df.groupby(['investment_id'])['target'].std()
# mean_std_target = np.mean(sts_target)
# ax = sns.jointplot(x=obs_by_asset.values, y=sts_target, kind="reg",height=8, joint_kws={'line_kws':{'color':'green'}})
# ax.ax_joint.set_xlabel('observations', size=20)
# ax.ax_joint.set_ylabel('std target', size=20)
# plt.show()

### Grid of scatter plot for each features

In [ ]:
# df1=df.iloc[:,2:27]
# g = sns.PairGrid(df1)
# g.map(sns.scatterplot)
# g.savefig('pairplot.jpg')

In [ ]:
# plt.figure(figsize=(20,20))
# for i in df1.columns:
#     sns.scatterplot(data=df,x=i,y='target',palette="deep")
#     g.map(sns.scatterplot)

# Modelling using light GBM

## Train Test Split

In [ ]:
# df1=df
# Y=df1.pop('target')
# X=df[['f_231', 'f_119', 'f_179', 'f_297', 'f_76']]
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42,shuffle=False)

## Implementing lightGBM

In [ ]:
# selection = ["time_id", "investment_id"]
# feature_names = df2.drop(['row_id',"time_id", "investment_id", "target"], axis=1).columns.values
# N = np.random.choice(np.arange(0, len(feature_names)), replace=False, size=10)
# my_features = ["f_{}".format(n) for n in N]
# selection.extend(my_features)
# X = df2[selection].copy()
# Y = df2.target
# for f in my_features:
#     X[f + "_shift1"] = X.groupby("investment_id")[f].shift(1).fillna(0)
# features = ["f_{}_shift1".format(n) for n in N]
# features.extend(["f_{}".format(n) for n in N])
# V = 0.9
# x_train, x_dev = X[0:int(V*X.shape[0])][features], X[int(V*X.shape[0])::][features]
# y_train, y_dev = Y[0:int(V*X.shape[0])], Y[int(V*X.shape[0])::]


In [ ]:
# lgb_train = lgb.Dataset(x_train.values, y_train.values)
# lgb_eval = lgb.Dataset(x_dev.values, y_dev.values, reference=lgb_train)
# params = {'boosting_type': 'gbdt',
#               'objective': 'regression',
#               'num_leaves': 5,
#               'learning_rate': 0.1,
#               'feature_fraction': 0.9
#               }
# lm = lgb.train(params,
#     lgb_train,
#     num_boost_round=10,
#     valid_sets=[lgb_train, lgb_eval],
#     valid_names=['train','valid'],
#     callbacks= [lgb.early_stopping(10)]
#    )
# y_pred = lm.predict(x_dev)
# pearsonr(y_dev, y_pred)

### DNN Implementation

In [ ]:
# class Config:
#     is_training = False
#     tf_record_dataset_path = "../input/ump-tf-record-combinatorialpurgedgroupkfold/"
#     output_dataset_path = "../input/ubiquant-market-prediction-with-dnn-output/"
# config = Config()

In [ ]:
# %%time
# investment_ids = pd.read_csv('../input/ump-tf-record-combinatorialpurgedgroupkfold/investment_ids.csv')
# investment_id_size = len(investment_ids) + 1
# with tf.device("cpu"):
#     investment_id_lookup_layer = layers.IntegerLookup(max_tokens=investment_id_size)
#     investment_id_lookup_layer.adapt(investment_ids)

In [ ]:
# def decode_function(record_bytes):
#   return tf.io.parse_single_example(
#       # Data
#       record_bytes,
#       # Schema
#       {
#           "features": tf.io.FixedLenFeature([300], dtype=tf.float32),
#           "time_id": tf.io.FixedLenFeature([], dtype=tf.int64),
#           "investment_id": tf.io.FixedLenFeature([], dtype=tf.int64),
#           "target": tf.io.FixedLenFeature([], dtype=tf.float32)
#       }
#   )
# def preprocess(item):
#     return (item["investment_id"], item["features"]), item["target"]
# def make_dataset(file_paths, batch_size=4096, mode="train"):
#     ds = tf.data.TFRecordDataset(file_paths)
#     ds = ds.map(decode_function)
#     ds = ds.map(preprocess)
#     if mode == "train":
#         ds = ds.shuffle(batch_size * 4)
#     ds = ds.batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)
#     return ds

In [ ]:
# def correlation(x, y, axis=-2):
#     x = tf.convert_to_tensor(x)
#     y = math_ops.cast(y, x.dtype)
#     n = tf.cast(tf.shape(x)[axis], x.dtype)
#     xsum = tf.reduce_sum(x, axis=axis)
#     ysum = tf.reduce_sum(y, axis=axis)
#     xmean = xsum / n
#     ymean = ysum / n
#     xvar = tf.reduce_sum( tf.math.squared_difference(x, xmean), axis=axis)
#     yvar = tf.reduce_sum( tf.math.squared_difference(y, ymean), axis=axis)
#     cov = tf.reduce_sum( (x - xmean) * (y - ymean), axis=axis)
#     corr = cov / tf.sqrt(xvar * yvar)
#     return tf.constant(1.0, dtype=x.dtype) - corr

# def get_model():
#     investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
#     features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
#     investment_id_x = investment_id_lookup_layer(investment_id_inputs)
#     investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
#     investment_id_x = layers.Reshape((-1, ))(investment_id_x)
#     investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
#     investment_id_x = layers.Dropout(0.1)(investment_id_x)
#     investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
#     investment_id_x = layers.Dropout(0.1)(investment_id_x)
#     investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
#     investment_id_x = layers.Dropout(0.1)(investment_id_x)
    
#     feature_x = layers.Dense(256, activation='swish')(features_inputs)
#     feature_x = layers.Dropout(0.1)(feature_x)
#     feature_x = layers.Dense(256, activation='swish')(feature_x)
#     feature_x = layers.Dropout(0.1)(feature_x)
#     feature_x = layers.Dense(256, activation='swish')(feature_x)
#     feature_x = layers.Dropout(0.1)(feature_x)
    
#     x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
#     x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(x)
#     x = layers.Dropout(0.1)(x)
#     x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
#     x = layers.Dropout(0.1)(x)
#     x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
#     x = layers.Dropout(0.1)(x)
#     output = layers.Dense(1)(x)
#     rmse = keras.metrics.RootMeanSquaredError(name="rmse")
#     model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
#     model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse, correlation])
#     return model

In [ ]:
# model = get_model()
# keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# %%time
# models = []
# for i in range(5):
#     train_path = f"{config.tf_record_dataset_path}fold_{i}_train.tfrecords"
#     valid_path = f"{config.tf_record_dataset_path}fold_{i}_test.tfrecords"
#     valid_ds = make_dataset([valid_path], mode="valid")
#     model = get_model()
#     if config.is_training:
#         train_ds = make_dataset([train_path])
#         checkpoint = keras.callbacks.ModelCheckpoint(f"model_{i}.tf", monitor="val_correlation", mode="min", save_best_only=True, save_weights_only=True)
#         early_stop = keras.callbacks.EarlyStopping(patience=10)
#         history = model.fit(train_ds, epochs=30, validation_data=valid_ds, callbacks=[checkpoint, early_stop])
#         model.load_weights(f"model_{i}.tf")
#         for metric in ["loss", "mae", "mape", "rmse", "correlation"]:
#             pd.DataFrame(history.history, columns=[metric, f"val_{metric}"]).plot()
#             plt.title(metric.upper())
#             plt.show()
#     else:
#         model.load_weights(f"{config.output_dataset_path}model_{i}.tf")
#     y_vals = []
#     for _, y in valid_ds:
#         y_vals += list(y.numpy().reshape(-1))
#     y_val = np.array(y_vals)
#     pearson_score = stats.pearsonr(model.predict(valid_ds).reshape(-1), y_val)[0]
#     models.append(model)
#     print(f"Pearson Score: {pearson_score}")

### model ensemble

In [ ]:
%%time
n_features = 300
features = [f'f_{i}' for i in range(n_features)]
feature_columns = ['investment_id', 'time_id'] + features
train = pd.read_parquet('../input/low-mem-parquet/train_low_mem.parquet')
train.head()

In [ ]:
investment_id = train.pop("investment_id")
_ = train.pop("time_id")
y = train.pop("target")

In [ ]:
investment_ids = list(investment_id.unique())
investment_id_size = len(investment_ids) + 1
investment_id_lookup_layer = layers.IntegerLookup(max_tokens=investment_id_size)
with tf.device("cpu"):
    investment_id_lookup_layer.adapt(investment_id)

In [ ]:
def preprocess(X, y):
    print(X)
    print(y)
    return X, y
def make_dataset(feature, investment_id, y, batch_size=1024, mode="train"):
    ds = tf.data.Dataset.from_tensor_slices(((investment_id, feature), y))
    ds = ds.map(preprocess)
    if mode == "train":
        ds = ds.shuffle(256)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [ ]:
def get_model():
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
    x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    output = layers.Dense(1)(x)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse])
    return model


def get_model2():
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)    
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
   # investment_id_x = layers.Dropout(0.65)(investment_id_x)
   
    
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dropout(0.65)(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
    x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(x)
   # x = layers.Dropout(0.2)(x)
    x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
  #  x = layers.Dropout(0.4)(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.75)(x)
    output = layers.Dense(1)(x)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse])
    return model


def get_model3():
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float32)
    
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dropout(0.5)(investment_id_x)
    investment_id_x = layers.Dense(32, activation='swish')(investment_id_x)
    investment_id_x = layers.Dropout(0.5)(investment_id_x)
    #investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dropout(0.5)(feature_x)
    feature_x = layers.Dense(128, activation='swish')(feature_x)
    feature_x = layers.Dropout(0.5)(feature_x)
    feature_x = layers.Dense(64, activation='swish')(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(16, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(1)(x)
    output = tf.keras.layers.BatchNormalization(axis=1)(output)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse])
    return model

def get_model5():
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
    ## feature ##
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dropout(0.1)(feature_x)
    ## convolution 1 ##
    feature_x = layers.Reshape((-1,1))(feature_x)
    feature_x = layers.Conv1D(filters=16, kernel_size=4, strides=1, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## convolution 2 ##
    feature_x = layers.Conv1D(filters=16, kernel_size=4, strides=4, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## convolution 3 ##
    feature_x = layers.Conv1D(filters=64, kernel_size=4, strides=1, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## convolution 4 ##
    feature_x = layers.Conv1D(filters=64, kernel_size=4, strides=4, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## convolution 5 ##
    feature_x = layers.Conv1D(filters=64, kernel_size=4, strides=2, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## flatten ##
    feature_x = layers.Flatten()(feature_x)
    
    x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(feature_x)
    
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.1)(x)
    output = layers.Dense(1)(x)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse])
    return model
del train
del investment_id
del y
gc.collect()

In [ ]:
# models = []

# for i in range(5):
#     model = get_model()
#     model.load_weights(f'../input/dnn-base/model_{i}')
#     models.append(model)

# for i in range(10):
#     model = get_model2()
#     model.load_weights(f'../input/train-dnn-v2-10fold/model_{i}')
#     models.append(model)
    
    
# for i in range(10):
#     model = get_model3()
#     model.load_weights(f'../input/dnnmodelnew/model_{i}')
#     models.append(model)
    
    
# models2 = []
    
# for i in range(5):
#     model = get_model5()
#     model.load_weights(f'../input/prediction-including-spatial-info-with-conv1d/model_{i}.tf')
#     models2.append(model)

In [ ]:
def preprocess_test(investment_id, feature):
    return (investment_id, feature), 0

def preprocess_test_s(feature):
    return (feature), 0

def make_test_dataset(feature, investment_id, batch_size=1024):
    ds = tf.data.Dataset.from_tensor_slices(((investment_id, feature)))
    ds = ds.map(preprocess_test)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    return ds

def make_test_dataset2(feature, batch_size=1024):
    ds = tf.data.Dataset.from_tensor_slices(((feature)))
    ds = ds.batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)
    return ds

def inference(models, ds):
    y_preds = []
    for model in models:
        y_pred = model.predict(ds)
        y_preds.append(y_pred)
    return np.mean(y_preds, axis=0)

from sklearn.decomposition import PCA
pca = PCA(n_components=1)


def make_test_dataset3(feature, batch_size=1024):
    ds = tf.data.Dataset.from_tensor_slices((feature))
    ds = ds.map(preprocess_test_s)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    return ds

def infer(models, ds):
    y_preds = []
    for model in models:
        y_pred = model.predict(ds)
        y_preds.append((y_pred-y_pred.mean())/y_pred.std())
    
    return np.mean(y_preds, axis=0)

In [ ]:
import ubiquant
env = ubiquant.make_env()
iter_test = env.iter_test() 
for (test_df, sample_prediction_df) in iter_test:
    ds = make_test_dataset(test_df[features], test_df["investment_id"])
    p1 = inference(models, ds)
    ds2 = make_test_dataset2(test_df[features])
    p2 = inference(models2, ds2)
    ds3 = make_test_dataset3(test_df[features])
    p3 = infer(models3, ds3)
    sample_prediction_df['target'] = p1 * 0.29 + p2 * 0.64 + p3 * 0.07
    env.predict(sample_prediction_df) 
    display(sample_prediction_df)

### Submission

In [ ]:
# def preprocess_test(investment_id, feature):
#     return (investment_id, feature), 0
# def make_test_dataset(feature, investment_id, batch_size=1024):
#     ds = tf.data.Dataset.from_tensor_slices(((investment_id, feature)))
#     ds = ds.map(preprocess_test)
#     ds = ds.batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)
#     return ds
# def inference(models, ds):
#     y_preds = []
#     for model in models:
#         y_pred = model.predict(ds)
#         y_preds.append(y_pred)
#     return np.mean(y_preds, axis=0)
# import ubiquant
# env = ubiquant.make_env()
# iter_test = env.iter_test() 
# features = [f"f_{i}" for i in range(300)]
# for (test_df, sample_prediction_df) in iter_test:
#     ds = make_test_dataset(test_df[features], test_df["investment_id"])
#     sample_prediction_df['target'] = inference(models, ds)
#     env.predict(sample_prediction_df) 

In [ ]:
# import ubiquant
# env = ubiquant.make_env()   # initialize the environment
# iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

# previous_test_df = df2[df2.time_id == df2.time_id.max()].iloc[0:int(V*df2.shape[0])]
# for (test_df, sample_prediction_df) in iter_test:
    
#     for f in my_features:
#         test_df.loc[:, f+"_shift1"] = 0
#         already_known = previous_test_df[previous_test_df.investment_id.isin(test_df.investment_id)]
#         test_df.loc[test_df.investment_id.isin(already_known.investment_id), f+"_shift1"] = already_known[f]
#         test_df.loc[:, f+"_shift1"] = test_df.loc[:, f+"_shift1"].fillna(0)
        
#         #test_df.loc[:, f+"_diff"] = test_df[f] - test_df[f+"_shift1"]
#         #test_df.loc[:, f+"_diff"] = test_df.loc[:, f+"_diff"].fillna(0)
    
#     pred = lm.predict(test_df[features])
#     sample_prediction_df['target'] = pred  # make your predictions here
#     env.predict(sample_prediction_df)   # register your predictions
#     previous_test_df = test_df.copy()

Submissions are scored on the mean of Pearson correlation coefficient for each time ID. Pearson correlation coefficient (Pearson's r) is a measure of linear correlation between two sets of values. It can be denoted as

r=n∑xy−(∑x)(∑y)[n∑x2−(∑x)2][n∑y2−(∑y)2−−−−−−−−−−−−−−−−−−−−−−−−−−√] 
r  = Pearson Correlation Coefficient
n  = Number of samples
x  = First set of values
y  = Second set of values
Mean of Pearson correlation coefficient across time IDs can be denoted as

Mean r=1T∑Ti=1tir 
T  = Number of time IDs
tir  = ith time ID's Pearson correlation coefficient